In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque
import matplotlib.pyplot as plt


from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D,LSTM,TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder

from keras.models import Model


import cv2


os.environ['KMP_DUPLICATE_LIB_OK']='True'

2025-05-22 16:37:44.385609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747931864.408899  221537 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747931864.416355  221537 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-22 16:37:44.441932: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_video_paths_and_categories(root_dir,classes,training_ratio=0.7,validation_ratio=0.15,test_ratio=0.15):
    training_video_paths = []
    training_targets = []
    validation_video_paths = []
    validation_targets = []
    test_video_paths = []
    test_targets = []
    
    label_encoder = LabelEncoder()
    label_encoder.fit(classes)  # Fit the encoder with the classes

    for category in classes:
        category_path = os.path.join(root_dir, category)
        
        if os.path.isdir(category_path):
            nvideos = len(os.listdir(category_path))
            if nvideos == 0:
                print(f"No videos found for category: {category}")
                continue

            # Split counts based on the ratios
            training_count = int(nvideos * training_ratio)
            validation_count = int(nvideos * validation_ratio)
            test_count = nvideos - (training_count + validation_count)  # Remaining videos for testing
            
            label = label_encoder.transform([category])[0]
            
            # Get video paths
            video_files = os.listdir(category_path)
            for i in range(training_count):
                video_file = video_files[i]
                video_path = os.path.join(category_path, video_file)
                training_video_paths.append(video_path)
                training_targets.append(label)

            for i in range(training_count, training_count + validation_count):
                video_file = video_files[i]
                video_path = os.path.join(category_path, video_file)
                validation_video_paths.append(video_path)
                validation_targets.append(label)

            for i in range(training_count + validation_count, training_count + validation_count + test_count):
                video_file = video_files[i]
                video_path = os.path.join(category_path, video_file)
                test_video_paths.append(video_path)
                test_targets.append(label)

        else:
            print(f"No class found for {category}")

    return (training_video_paths, training_targets, 
            validation_video_paths, validation_targets, 
            test_video_paths, test_targets)

In [5]:
#get frames from video

def get_frames_from_video(video_path,frame_shape, n_frames=10):
    cap = cv2.VideoCapture(video_path)
        
    if not cap.isOpened():
        print(f"Error: Could not open video at path {video_path}.")
        return None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        print("Error: Video contains no frames.")
        cap.release()
        return None

    step = max(1, total_frames // n_frames)
    frames_count = 0
    result = []

    index = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Select frames based on step
        if index % step == 0 and frames_count < n_frames:
            resized_frame = cv2.resize(frame, frame_shape)
            result.append(resized_frame)
            frames_count += 1
        
        index += 1

    cap.release()
        
    while frames_count < n_frames:
        # print('aici')
        for i in range(1,n_frames,1):
            if frames_count < n_frames:
                frames_count+=1
                result.append(result[-i])
        
    result = np.stack(result, axis=0)
    result = result.astype(np.float32) / 255.0  # Normalize to [0, 1]

    return result
    
    
    

In [3]:
dataset_dir='../../UCF50'
image_shape=(128,128)
number_of_frames=15

training_ratio=0.6
validation_ratio=0.25
test_ratio=0.15

classes =['PullUps','Punch','PushUps']
num_classes = len(classes)

In [4]:
def extract_ucf50_classes(dataset_dir):
    classes = [folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))]
    return classes


classes = extract_ucf50_classes(dataset_dir)
num_classes = len(classes)

label_encoder=LabelEncoder()
label_encoder.fit(classes)

# classes = classes[:3]
# num_classes = len(classes)

LabelEncoder()

In [ ]:
import joblib

joblib.dump(label_encoder, 'label_encoder.pkl')



['label_encoder.pkl']

In [6]:
x_train, y_train, x_val, y_val, x_test, y_test = get_video_paths_and_categories(dataset_dir,classes,training_ratio,validation_ratio,test_ratio)

In [7]:
x_train_data = np.array([get_frames_from_video(video_path, image_shape, number_of_frames) for video_path in x_train])
x_val_data = np.array([get_frames_from_video(video_path, image_shape, number_of_frames) for video_path in x_val])
x_test_data = np.array([get_frames_from_video(video_path, image_shape, number_of_frames) for video_path in x_test])

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

KeyboardInterrupt: 

In [10]:
np.save('../x_train_data.npy', x_train_data)
np.save('../x_val_data.npy', x_val_data)
np.save('../x_test_data.npy', x_test_data)
np.save('../y_train.npy', y_train)
np.save('../y_val.npy', y_val)
np.save('../y_test.npy', y_test)

In [2]:
x_train_data = np.load('../x_train_data.npy')
x_val_data = np.load('../x_val_data.npy')
x_test_data = np.load('../x_test_data.npy')
y_train = np.load('../y_train.npy')
y_val = np.load('../y_val.npy') 
y_test = np.load('../y_test.npy')

In [3]:
x_train_data.shape, x_val_data.shape, x_test_data.shape, y_train.shape, y_val.shape, y_test.shape

((3992, 15, 128, 128, 3),
 (1655, 15, 128, 128, 3),
 (1034, 15, 128, 128, 3),
 (3992,),
 (1655,),
 (1034,))

In [ ]:
from tensorflow.keras.applications import ResNet152V2,ResNet50V2
from tensorflow.keras.layers import Input, LSTM, TimeDistributed, Dense, Flatten, Concatenate, GlobalAveragePooling1D,Reshape,ConvLSTM2D,Conv3D,MaxPooling3D

def create_model(input_shape, num_classes, number_of_frames=10):
    
    pretrained_model = ResNet50V2(
        include_top=False,
        weights="imagenet",
        pooling="avg",
        input_shape=input_shape[1:]
    )
    pretrained_model.trainable = False 

    
    video_input = Input(shape=input_shape, name="video_input")
    x = TimeDistributed(pretrained_model)(video_input)
    
  
    x = LSTM(512, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(256, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(256, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(256)(x)

    final_output = Dense(num_classes, activation="softmax")(x)
    
    model = Model(inputs=video_input, outputs=final_output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [13]:
model=create_model(x_train_data.shape[1:], num_classes, number_of_frames)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ video_input (InputLayer)        │ (None, 15, 128, 128,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 2048)       │    23,564,800 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 15, 512)        │     5,244,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 15, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 15, 256)        │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 15, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_16 (LSTM)                  │ (None, 15, 256)        │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 15, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 15, 128)        │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_18 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         6,450 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,457,650 (116.19 MB)

 Trainable params: 6,892,850 (26.29 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [5]:

from keras.callbacks import ReduceLROnPlateau

early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True, 

    verbose=1
)

checkpoint_callback = ModelCheckpoint(
    filepath='resnet50_lstm_proba.keras',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),



In [6]:
x_train_data.shape

(3992, 15, 128, 128, 3)

In [10]:
model=create_model(x_train_data.shape[1:], num_classes, number_of_frames)

model.fit(
    x_train_data, y_train,
    validation_data=(x_val_data, y_val),
    epochs=500,
    batch_size=32,
    callbacks=[early_stopping_callback, checkpoint_callback,reduce_lr],
    verbose=1
)

W0000 00:00:1742230182.641538 4140594 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/500


2025-03-17 16:49:48.891122: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 11772887040 exceeds 10% of free system memory.


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3521 - loss: 2.6727
Epoch 1: val_accuracy improved from -inf to 0.56133, saving model to resnet50_lstm_proba.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 419s 3s/step - accuracy: 0.3530 - loss: 2.6680 - val_accuracy: 0.5613 - val_loss: 1.5472 - learning_rate: 0.0010
Epoch 2/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6645 - loss: 1.2227
Epoch 2: val_accuracy improved from 0.56133 to 0.67372, saving model to resnet50_lstm_proba.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 333s 3s/step - accuracy: 0.6647 - loss: 1.2220 - val_accuracy: 0.6737 - val_loss: 1.1538 - learning_rate: 0.0010
Epoch 3/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7833 - loss: 0.7637
Epoch 3: val_accuracy improved from 0.67372 to 0.74622, saving model to resnet50_lstm_proba.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 331s 3s/step - accuracy: 0.7833 - loss: 0.7638 - val_accuracy: 0.7462 - val_loss: 0.8805 - learning_rate: 0.0010
Epoch 4/500
125/125 ━━━━━━━━━━━━━━━━━

In [ ]:
model=keras.models.load_model('resnet50_lstm_proba.keras')
# model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ video_input (InputLayer)        │ (None, 15, 128, 128,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_26             │ (None, 15, 2048)       │    23,564,800 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_26 (LSTM)                  │ (None, 15, 512)        │     5,244,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 15, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_27 (LSTM)                  │ (None, 15, 256)        │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 15, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_28 (LSTM)                  │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 50)             │        12,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,276,440 (165.09 MB)

 Trainable params: 6,570,546 (25.06 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

 Optimizer params: 13,141,094 (50.13 MB)

In [3]:
model=keras.models.load_model('resnet50_lstm.keras')
accuracy,loss=model.evaluate(x_test_data,y_test)
print(f"Accuracy: {accuracy}")
print(f"Loss: {loss}")

W0000 00:00:1747932022.097310  221537 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-05-22 16:40:47.142927: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3049390080 exceeds 10% of free system memory.


33/33 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.8320 - loss: 0.8531
Accuracy: 0.6354348659515381
Loss: 0.8636363744735718


In [ ]:
#best

model=keras.models.load_model('resnet50_lstm.keras')
accuracy,loss=model.evaluate(x_test_data,y_test)
print(f"Accuracy: {accuracy}")
print(f"Loss: {loss}")

W0000 00:00:1744182931.942411   99943 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-04-09 07:15:33.887405: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3049390080 exceeds 10% of free system memory.


33/33 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.8320 - loss: 0.8531
Accuracy: 0.6354348659515381
Loss: 0.8636363744735718


In [11]:

model.evaluate(x_test_data, y_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.8237 - loss: 0.6824


[0.6778411269187927, 0.823017418384552]

In [70]:
#best 

model.evaluate(x_test_data, y_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.8320 - loss: 0.8531


[0.6354348659515381, 0.8636363744735718]